<a href="https://colab.research.google.com/github/ShreyasJothish/DS-Unit-2-Sprint-2-Linear-Regression/blob/master/module2-polynomial-regression/Polynomial_Log_linear_Regression_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Intermediate Linear Regression Practice

## Use a Linear Regression model to get the lowest RMSE possible on the following dataset:

[Dataset Folder](https://github.com/ryanleeallred/datasets/tree/master/Ames%20Housing%20Data)

[Raw CSV](https://raw.githubusercontent.com/ryanleeallred/datasets/master/Ames%20Housing%20Data/train.csv)

## You model must include (at least):
- A log-transformed y variable
- Two polynomial features
- One interaction feature
- 10 other engineered features

What is the lowest Root-Mean-Squared Error that you are able to obtain? Share your best RMSEs in Slack!

Notes:

There may be some data cleaning that you need to do on some features of this dataset. Linear Regression will only accept numeric values and will not accept

Note* There may not be a clear candidate for an interaction term in this dataset. Include one anyway, sometimes it's a good practice for predictive modeling feature engineering in general. 

In [1]:
!pip install -U seaborn

Requirement already up-to-date: seaborn in /usr/local/lib/python3.6/dist-packages (0.9.0)


In [0]:
# Imports all in one place
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

In [0]:
# Data clean up and using only Ordinal and Numerical features
dataurl = "https://raw.githubusercontent.com/ryanleeallred/datasets/master/Ames%20Housing%20Data/train.csv"
df = pd.read_csv(dataurl)

In [4]:
pd.set_option('display.max_rows', 100)
print(df.shape)
print(df.columns)

(1460, 81)
Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 

In [5]:
df_num = df[["LotFrontage",
"LotArea",
"OverallQual",
"OverallCond",
"BsmtFinSF1",
"BsmtFinSF2",
"BsmtUnfSF",
"TotalBsmtSF",
"1stFlrSF",
"2ndFlrSF",
"LowQualFinSF",
"GrLivArea",
"BsmtFullBath",
"BsmtHalfBath",
"FullBath",
"HalfBath",
"TotRmsAbvGrd",
"Fireplaces",
"GarageCars",
"GarageArea",
"WoodDeckSF",
"OpenPorchSF",
"EnclosedPorch",
"3SsnPorch",
"ScreenPorch",
"PoolArea"]].copy()
df_num.sample(5)

,LotFrontage,LotArea,OverallQual,OverallCond,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,...,TotRmsAbvGrd,Fireplaces,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea
229,43.0,3182,7,5,16,0,1357,1373,1555,0,...,7,1,2,430,143,20,0,0,0,0
759,65.0,12257,8,5,56,64,1198,1318,1328,1203,...,9,1,3,752,222,98,0,0,0,0
396,60.0,7200,5,5,777,0,117,894,894,0,...,6,0,2,600,215,0,0,0,0,0
289,60.0,8730,6,7,0,0,698,698,698,698,...,7,0,1,384,0,0,0,0,259,0
1183,60.0,10800,5,6,821,0,299,1120,1130,0,...,5,1,2,720,229,0,0,0,0,0


In [6]:
df_num.isnull().sum()

LotFrontage      259
LotArea            0
OverallQual        0
OverallCond        0
BsmtFinSF1         0
BsmtFinSF2         0
BsmtUnfSF          0
TotalBsmtSF        0
1stFlrSF           0
2ndFlrSF           0
LowQualFinSF       0
GrLivArea          0
BsmtFullBath       0
BsmtHalfBath       0
FullBath           0
HalfBath           0
TotRmsAbvGrd       0
Fireplaces         0
GarageCars         0
GarageArea         0
WoodDeckSF         0
OpenPorchSF        0
EnclosedPorch      0
3SsnPorch          0
ScreenPorch        0
PoolArea           0
dtype: int64

In [7]:
LotFrontage_mean = df_num.LotFrontage.mean()
#df_num.loc[:,'LotFrontage'].fillna(LotFrontage_mean, inplace=True)
df_num.LotFrontage.fillna(df_num.LotFrontage.mean(), inplace=True)

# Rechecking
df_num.isnull().sum()

LotFrontage      0
LotArea          0
OverallQual      0
OverallCond      0
BsmtFinSF1       0
BsmtFinSF2       0
BsmtUnfSF        0
TotalBsmtSF      0
1stFlrSF         0
2ndFlrSF         0
LowQualFinSF     0
GrLivArea        0
BsmtFullBath     0
BsmtHalfBath     0
FullBath         0
HalfBath         0
TotRmsAbvGrd     0
Fireplaces       0
GarageCars       0
GarageArea       0
WoodDeckSF       0
OpenPorchSF      0
EnclosedPorch    0
3SsnPorch        0
ScreenPorch      0
PoolArea         0
dtype: int64

In [8]:
df_ord = df[["ExterQual",
"ExterCond",
"BsmtQual",
"BsmtCond",
"BsmtExposure",
"HeatingQC",
"CentralAir",
"KitchenQual",
"FireplaceQu",
"GarageQual",
"GarageCond",
"PoolQC",
"Fence"]].copy()
df_ord.isnull().sum()

ExterQual          0
ExterCond          0
BsmtQual          37
BsmtCond          37
BsmtExposure      38
HeatingQC          0
CentralAir         0
KitchenQual        0
FireplaceQu      690
GarageQual        81
GarageCond        81
PoolQC          1453
Fence           1179
dtype: int64

In [0]:
# Dropping FireplaceQu, PoolQC and Fence since most of the values are NaN
df_ord.drop(columns=["FireplaceQu", "PoolQC", "Fence"], inplace=True)
df_ord.fillna(method='backfill', inplace=True)

In [10]:
!pip install category_encoders

In [11]:
import category_encoders as ce

encoder = ce.binary.BinaryEncoder()

df_ord_binenc = encoder.fit_transform(df_ord)
df_ord_binenc.head()

,ExterQual_0,ExterQual_1,ExterQual_2,ExterCond_0,ExterCond_1,ExterCond_2,ExterCond_3,BsmtQual_0,BsmtQual_1,BsmtQual_2,...,KitchenQual_1,KitchenQual_2,GarageQual_0,GarageQual_1,GarageQual_2,GarageQual_3,GarageCond_0,GarageCond_1,GarageCond_2,GarageCond_3
0,0,0,1,0,0,0,1,0,0,1,...,0,1,0,0,0,1,0,0,0,1
1,0,1,0,0,0,0,1,0,0,1,...,1,0,0,0,0,1,0,0,0,1
2,0,0,1,0,0,0,1,0,0,1,...,0,1,0,0,0,1,0,0,0,1
3,0,1,0,0,0,0,1,0,1,0,...,0,1,0,0,0,1,0,0,0,1
4,0,0,1,0,0,0,1,0,0,1,...,0,1,0,0,0,1,0,0,0,1


In [12]:
X = df_num.join(df_ord_binenc)

print(df_num.shape)
print(df_ord_binenc.shape)
print(X.shape)

(1460, 26)
(1460, 33)
(1460, 59)


In [0]:
# log-transformed y variable
y = np.log(df['SalePrice'])

In [16]:
# Slit into test and train datasets
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=.5, random_state=42)

# fit model using train datasets
model = LinearRegression()
model.fit(X_train, Y_train)

# Create new predictions using x_test
y_pred = model.predict(X_test)

# Measure Accuracy using y_test and y_pred
RMSE = (np.sqrt(mean_squared_error(Y_test, y_pred)))
R2 = r2_score(Y_test, y_pred)

print('RMSE is {}'.format(RMSE))
print('R^2 is {}'.format(R2))

print("coefficients: ", model.coef_)
print("intercepts: ", model.intercept_)

RMSE is 0.16149132010191228
R^2 is 0.8436072152250504
coefficients:  [-1.14992303e-04  2.10578812e-06  7.22529379e-02  3.18903424e-02
 -5.44710472e-06  3.22131675e-05 -1.54067068e-05  1.13593526e-05
  9.56828371e-05 -5.98843065e-07  1.76357569e-05  1.12719753e-04
  5.41756542e-02  3.47262367e-02  5.50502018e-02  6.00141961e-02
  8.78978945e-03  3.16365241e-02  1.00450554e-01  1.60196619e-06
  7.30952476e-05  6.60468545e-05  1.27752933e-04  4.75912497e-04
  2.88615925e-04 -7.96153598e-04 -3.20108102e-03 -1.65041106e-02
  3.64070393e-02 -2.08166817e-17 -4.71230236e-02  2.67604409e-02
  4.64551736e-02  1.86757691e-02  6.80117332e-02  1.14065559e-01
 -1.00567382e-01 -7.68652764e-02 -9.46220079e-02 -2.85542722e-02
  1.23807033e-02 -4.49685337e-02  3.46944695e-17 -6.78024505e-02
 -3.33726208e-02 -9.09741565e-03 -7.02836940e-02  7.02836940e-02
  9.73378309e-02  6.00509573e-02  1.08079177e-01  0.00000000e+00
  4.40698560e-03  7.89987059e-02  1.16637895e-01  0.00000000e+00
  1.93730266e-01  1.0

In [0]:
# Two polynomial features
X['OverallQual_Sq'] = X['OverallQual'] ** 2
X['OverallCond_Sq'] = X['OverallCond'] ** 2

In [19]:
# Slit into test and train datasets
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=.5, random_state=42)

# fit model using train datasets
model = LinearRegression()
model.fit(X_train, Y_train)

# Create new predictions using x_test
y_pred = model.predict(X_test)

# Measure Accuracy using y_test and y_pred
RMSE = (np.sqrt(mean_squared_error(Y_test, y_pred)))
R2 = r2_score(Y_test, y_pred)

print('RMSE is {}'.format(RMSE))
print('R^2 is {}'.format(R2))

print("coefficients: ", model.coef_)
print("intercepts: ", model.intercept_)

RMSE is 0.16085205781477196
R^2 is 0.844842924141786
coefficients:  [-7.89674965e-05  2.09726351e-06  1.12579123e-01  1.03538806e-01
 -6.16304952e-06  2.92748983e-05 -1.55188770e-05  7.59296883e-06
  1.04283167e-04  4.07386220e-06  5.07354412e-06  1.13430571e-04
  5.44853848e-02  3.37299919e-02  5.23142107e-02  5.79340146e-02
  8.17822671e-03  3.01373114e-02  9.95073120e-02  8.09474051e-06
  8.28442896e-05  6.57092142e-05  1.19232662e-04  4.67350640e-04
  2.79401371e-04 -7.94080598e-04  1.54833883e-02 -9.04568597e-04
  5.37364293e-02 -1.14491749e-16  2.39281582e-02  3.30515204e-02
  4.53752322e-02  3.06275808e-02  7.24925840e-02  1.15665820e-01
  9.29124752e-02 -6.63058484e-02 -8.28372554e-02 -2.89734103e-02
  1.22901455e-02 -4.52249551e-02 -4.16333634e-17 -6.32936969e-02
 -3.33852972e-02 -9.74895402e-03 -6.76556943e-02  6.76556943e-02
  1.16231163e-01  6.97952070e-02  1.21784194e-01 -1.38777878e-17
  1.01981938e-01  7.99102048e-02  1.16142764e-01  0.00000000e+00
  8.30806984e-02  9.90

In [0]:
# One interaction feature
X['TotRmsAbvGrd_Fireplaces'] = X['TotRmsAbvGrd'] * X['Fireplaces']

In [21]:
# Slit into test and train datasets
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=.5, random_state=42)

# fit model using train datasets
model = LinearRegression()
model.fit(X_train, Y_train)

# Create new predictions using x_test
y_pred = model.predict(X_test)

# Measure Accuracy using y_test and y_pred
RMSE = (np.sqrt(mean_squared_error(Y_test, y_pred)))
R2 = r2_score(Y_test, y_pred)

print('RMSE is {}'.format(RMSE))
print('R^2 is {}'.format(R2))

print("coefficients: ", model.coef_)
print("intercepts: ", model.intercept_)

RMSE is 0.16028549616134788
R^2 is 0.8459340041932921
coefficients:  [-1.02499545e-04  2.18874643e-06  9.53297459e-02  9.99054358e-02
 -3.28339990e-06  3.82878068e-05 -2.02527661e-05  1.47516466e-05
  1.15021501e-04  2.04357496e-05 -2.72880178e-05  1.08169227e-04
  5.04168625e-02  2.61519007e-02  5.24684669e-02  5.76190157e-02
  1.62844719e-02  1.31777510e-01  1.00323568e-01  3.41376910e-06
  7.42369203e-05  6.22022701e-05  9.64658828e-05  4.67386160e-04
  2.72797647e-04 -6.82362856e-04  2.00221391e-02  2.77408812e-03
  5.46905807e-02 -1.00613962e-16  3.03840559e-02  4.10941670e-02
  5.48404498e-02  3.38690441e-02  7.15261902e-02  1.16471908e-01
  6.25426121e-02 -7.17904299e-02 -9.32692500e-02 -2.54276433e-02
  1.35901372e-02 -4.21133607e-02  1.11022302e-16 -5.85030766e-02
 -3.01824788e-02 -7.02910146e-03 -6.72524180e-02  6.72524180e-02
  1.16381334e-01  7.33043861e-02  1.29669185e-01  4.85722573e-17
  1.79675470e-01  6.16315389e-02  9.02850842e-02  0.00000000e+00
 -2.43046319e-02  9.5

In [0]:
# 10 other engineered features

# Stretch Goals

- Write a blog post explaining one of today's topics.
- Find a new regression dataset from the UCI machine learning repository and use it to test out your new modeling skillz.
 [ - UCI Machine Learning Repository - Regression Datasets](https://)
- Make a list for yourself of common feature engineering techniques. Browse Kaggle kernels to learn more methods.
- Start studying for tomorrow's topic: Gradient Descent
- Try and make the ultimate model with this dataset. clean as many features as possible, engineer the most sensible features as possible and see how accurate of a prediction you can make. 
- Learn about the "Dummy Variable Trap" and how it applies to linear regression modeling.
- Learning about using linear regression to model time series data